# Recommendation Engine by David Ikhu-Omoregbee

In [2]:
import pandas as pd
import pyodbc
import numpy as np
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
import implicit


# Creating connection between Python an Sql server

In [3]:
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-9423IQKM\SQLEXPRESS;"
                      "Database=AdventureWorksDW2017;"
                      "Trusted_Connection=yes;")

# Quering the DW database

In [4]:
#Import Factinternetsales from AHG SQL Datawarehouse
fctintsales = pd.read_sql_query("""
SELECT
[CustomerKey],
[ProductKey],
[OrderQuantity],
[SalesAmount],
[OrderDate],
[SalesTerritoryKey],
[SalesOrderNumber]
FROM [dbo].[FactInternetSales]""", conn)

In [5]:
fctintsales.head(n =10) # Viewing the first 10 rows on Factinternetsales table

,CustomerKey,ProductKey,OrderQuantity,SalesAmount,OrderDate,SalesTerritoryKey,SalesOrderNumber
0,21768,310,1,3578.2700,2010-12-29,6,SO43697
1,28389,346,1,3399.9900,2010-12-29,7,SO43698
2,25863,346,1,3399.9900,2010-12-29,1,SO43699
3,14501,336,1,699.0982,2010-12-29,4,SO43700
4,11003,346,1,3399.9900,2010-12-29,9,SO43701
5,27645,311,1,3578.2700,2010-12-30,4,SO43702
6,16624,310,1,3578.2700,2010-12-30,9,SO43703
7,11005,351,1,3374.9900,2010-12-30,9,SO43704
8,11011,344,1,3399.9900,2010-12-30,9,SO43705
9,27621,312,1,3578.2700,2010-12-31,4,SO43706


In [6]:
fctintsales.tail(n = 10) # viewin the last 10 rows  of the Factinternetsales table

,CustomerKey,ProductKey,OrderQuantity,SalesAmount,OrderDate,SalesTerritoryKey,SalesOrderNumber
60388,18749,491,1,53.99,2014-01-28,6,SO75120
60389,18749,225,1,8.99,2014-01-28,6,SO75120
60390,15251,528,1,4.99,2014-01-28,6,SO75121
60391,15251,537,1,35.00,2014-01-28,6,SO75121
60392,15251,214,1,34.99,2014-01-28,6,SO75121
60393,15868,485,1,21.98,2014-01-28,6,SO75122
60394,15868,225,1,8.99,2014-01-28,6,SO75122
60395,18759,485,1,21.98,2014-01-28,6,SO75123
60396,18759,486,1,159.00,2014-01-28,6,SO75123
60397,18759,225,1,8.99,2014-01-28,6,SO75123


In [7]:
fctintsales.info() # displays datatypes , range and non-null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CustomerKey        60398 non-null  int64         
 1   ProductKey         60398 non-null  int64         
 2   OrderQuantity      60398 non-null  int64         
 3   SalesAmount        60398 non-null  float64       
 4   OrderDate          60398 non-null  datetime64[ns]
 5   SalesTerritoryKey  60398 non-null  int64         
 6   SalesOrderNumber   60398 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 3.2+ MB


In [8]:
fctintsales.describe() # stastical analysis of numerical columns

,CustomerKey,ProductKey,OrderQuantity,SalesAmount,SalesTerritoryKey
count,60398.000000,60398.000000,60398.0,60398.000000,60398.000000
mean,18841.685420,437.557932,1.0,486.086911,6.244462
std,5432.430404,118.088390,0.0,928.489892,2.961150
min,11000.000000,214.000000,1.0,2.290000,1.000000
25%,14003.000000,359.000000,1.0,7.950000,4.000000
50%,18143.000000,479.000000,1.0,29.990000,7.000000
75%,23429.750000,529.000000,1.0,539.990000,9.000000
max,29483.000000,606.000000,1.0,3578.270000,10.000000


# Using existing connection, import DimProductTable

In [9]:
# quering DimProductTable
product_tbl = pd.read_sql_query("""
SELECT
ProductKey,
EnglishProductName AS Description
FROM [dbo].[DimProduct]""", conn)


In [10]:
product_tbl.head() # viewing first 5 rows

,ProductKey,Description
0,1,Adjustable Race
1,2,Bearing Ball
2,3,BB Ball Bearing
3,4,Headset Ball Bearings
4,5,Blade


In [11]:
product_tbl.tail() # viewing last 5 rows

,ProductKey,Description
601,602,ML Bottom Bracket
602,603,HL Bottom Bracket
603,604,"Road-750 Black, 44"
604,605,"Road-750 Black, 48"
605,606,"Road-750 Black, 52"


In [12]:
product_tbl.info()
product_tbl.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ProductKey   606 non-null    int64 
 1   Description  606 non-null    object
dtypes: int64(1), object(1)
memory usage: 9.6+ KB


,ProductKey
count,606.00000
mean,303.50000
std,175.08141
min,1.00000
25%,152.25000
50%,303.50000
75%,454.75000
max,606.00000


In [13]:
# Using existing connection to join  all required tables

In [14]:
fis_pd = pd.read_sql_query("""
SELECT
fis.[CustomerKey] as CustomerID,
fis.[ProductKey] as StockCode,
fis.[OrderQuantity] as Quantity,
fis.[SalesAmount],
fis.[OrderDate],
fis.[SalesOrderNumber] ,
pd.EnglishProductName AS Description
FROM [dbo].[FactInternetSales] as fis
left join [dbo].[DimProduct] as Pd
on fis.[ProductKey] = pd.[ProductKey]""", conn)

In [15]:
fis_pd.head() # viewing first 5 rows

,CustomerID,StockCode,Quantity,SalesAmount,OrderDate,SalesOrderNumber,Description
0,21768,310,1,3578.2700,2010-12-29,SO43697,"Road-150 Red, 62"
1,28389,346,1,3399.9900,2010-12-29,SO43698,"Mountain-100 Silver, 44"
2,25863,346,1,3399.9900,2010-12-29,SO43699,"Mountain-100 Silver, 44"
3,14501,336,1,699.0982,2010-12-29,SO43700,"Road-650 Black, 62"
4,11003,346,1,3399.9900,2010-12-29,SO43701,"Mountain-100 Silver, 44"


In [16]:
fis_pd.tail()  # viewing last 5 rows 

,CustomerID,StockCode,Quantity,SalesAmount,OrderDate,SalesOrderNumber,Description
60393,15868,485,1,21.98,2014-01-28,SO75122,Fender Set - Mountain
60394,15868,225,1,8.99,2014-01-28,SO75122,AWC Logo Cap
60395,18759,485,1,21.98,2014-01-28,SO75123,Fender Set - Mountain
60396,18759,486,1,159.00,2014-01-28,SO75123,All-Purpose Bike Stand
60397,18759,225,1,8.99,2014-01-28,SO75123,AWC Logo Cap


In [17]:
fis_pd.info() # displays datatypes  for each columns, range and non-null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CustomerID        60398 non-null  int64         
 1   StockCode         60398 non-null  int64         
 2   Quantity          60398 non-null  int64         
 3   SalesAmount       60398 non-null  float64       
 4   OrderDate         60398 non-null  datetime64[ns]
 5   SalesOrderNumber  60398 non-null  object        
 6   Description       60398 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 3.2+ MB


In [18]:
cleaned_fis_pd = fis_pd.loc[pd.isnull(fis_pd.CustomerID) == False]

In [19]:
cleaned_fis_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60398 entries, 0 to 60397
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CustomerID        60398 non-null  int64         
 1   StockCode         60398 non-null  int64         
 2   Quantity          60398 non-null  int64         
 3   SalesAmount       60398 non-null  float64       
 4   OrderDate         60398 non-null  datetime64[ns]
 5   SalesOrderNumber  60398 non-null  object        
 6   Description       60398 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 3.7+ MB


In [20]:
cleaned_fis_pd["StockCode"] = cleaned_fis_pd["StockCode"].apply(str)

In [21]:
cleaned_fis_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60398 entries, 0 to 60397
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CustomerID        60398 non-null  int64         
 1   StockCode         60398 non-null  object        
 2   Quantity          60398 non-null  int64         
 3   SalesAmount       60398 non-null  float64       
 4   OrderDate         60398 non-null  datetime64[ns]
 5   SalesOrderNumber  60398 non-null  object        
 6   Description       60398 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 3.7+ MB


##### Before we make any sort of ratings matrix, it would be nice to have a
##### lookup table that keeps track of each item ID along with a 
###### description of that item

In [22]:

item_lookup = cleaned_fis_pd[['StockCode', 'Description']].drop_duplicates() # Only get unique item/description pairs
item_lookup['StockCode'] = item_lookup.StockCode.astype(str)

In [23]:
item_lookup.head()

,StockCode,Description
0,310,"Road-150 Red, 62"
1,346,"Mountain-100 Silver, 44"
3,336,"Road-650 Black, 62"
5,311,"Road-150 Red, 44"
7,351,"Mountain-100 Black, 48"


##### This last step is especially important if you don’t want to have unnecessary memory issues!
##### If you think about it, our matrix is going to contain thousands of items and thousands
##### of users with a user/item value required for every possible combination. 
##### That is a LARGE matrix, so we can save a lot of memory by keeping the matrix sparse and only
##### saving the locations and values of items that are not zero.
##### The code below will finish the preprocessing steps necessary for our final ratings sparse matrix:

In [24]:
#cleaned_retail['CustomerID'] = cleaned_retail.CustomerID.astype(int) # Convert to int for customer ID
cleaned_fis_pd = cleaned_fis_pd[['StockCode', 'Quantity', 'CustomerID']] # Get rid of unnecessary info
grouped_cleaned= cleaned_fis_pd.groupby(['CustomerID', 'StockCode']).sum().reset_index() # Group together
grouped_cleaned.Quantity.loc[grouped_cleaned.Quantity == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('Quantity > 0') # Only get customers where purchase totals were positive

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


##### If we look at our final resulting matrix of grouped purchases, we see the following:

In [63]:
grouped_purchased.head(n=10)

,CustomerID,StockCode,Quantity
0,11000,214,1
1,11000,344,1
2,11000,353,1
3,11000,485,1
4,11000,488,1
5,11000,530,1
6,11000,541,1
7,11000,573,1
8,11001,217,1
9,11001,225,1


##### Instead of representing an explicit rating, the purchase quantity can represent a “confidence” in terms of  how strong the interaction was. 
##### Items with a larger number of purchases by a customer can carry more weight in our ratings matrix of purchases.
##### Our last step is to create the sparse ratings matrix of users and items utilizing the code below:

In [64]:
from pandas.api.types import CategoricalDtype


customers = list(np.sort(grouped_purchased.CustomerID.unique())) # Get our unique customers
products = list(grouped_purchased.StockCode.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases

rows = grouped_purchased.CustomerID.astype(CategoricalDtype(categories = customers)).cat.codes
# Get the associated row indicescategories = customers).cat.codes
cols = grouped_purchased.StockCode.astype(CategoricalDtype(categories = products)).cat.codes
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))


##### Let’s check our final matrix object:

In [65]:
purchases_sparse

<18484x158 sparse matrix of type '<class 'numpy.intc'>'
	with 59051 stored elements in Compressed Sparse Row format>

##### We have 18484 customers with 158 items. For these user/item interactions,
##### 59051 of these items had a purchase. 
##### In terms of sparsity of the matrix, that makes:

In [27]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

97.97803231806365

##### 97.97% of the interaction matrix is sparse. For collaborative filtering to work,
##### the maximum sparsity you could get away with would probably be about 99.5% or so.
##### We are well below this, so we should be able to get decent results.

In [28]:
#from sklearn.model_selection
from sklearn.model_selection import RandomizedSearchCV

In [29]:
import random

In [30]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

###### This will return our training set, a test set that has been binarized to 0/1 for purchased/not purchased, and a  list of which users had at least one item masked. We will test the performance of the recommender system on these users only. I am masking 20% of the user/item interactions for this example.

In [31]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [32]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

##### Implementing ALS for Implicit Feedback

In [33]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 


In [34]:
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1,
                                            rank_size = 20)

##### We can investigate ratings for a particular user by taking the dot product between the user and item vectors (U and V). Let’s look at our first user.


In [35]:
user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

array([5.24299219e-02, 9.85570586e-05, 1.25361284e-02, 3.89815739e-02,
       7.83943702e-03])

##### This is a sample of the first five items out of the 154 in our stock. 
##### The first user in our matrix has the fifth item with the greatest recommendation out of the first five items. However, notice we only did one iteration because our algorithm was so slow! 
##### You should iterate at least ten times

## Speeding Up ALS

In [36]:
import implicit

In [37]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


## Evaluating the Recommender System
##### let’s make a simple function that can calculate our AUC. Scikit-learn has one we can alter a bit.

In [38]:
from sklearn import metrics

In [39]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   

In [40]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

##### We can now use this function to see how our recommender system is doing. 
##### To use this function, we will need to transform our output from the ALS function to csr_matrix format and transpose the item vectors

In [41]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.772, 0.836)

##### We can see that our recommender system beat popularity. Our system had a mean AUC of 0.772, while the popular item benchmark had a lower AUC of 0.836. You can go back and tune the hyperparameters if you wish to see if you can get a higher AUC score. Ideally, you would have separate train, cross-validation, and test sets so that you aren’t overfitting while tuning the hyperparameters, but this setup is adequate to demonstrate how to check that the system is working.

##### We now have our recommender system trained and have proven it beats the benchmark of popularity. An AUC of 0.772 means the system is recommending items the user in fact had purchased in the test set far more frequently than items the user never ended up purchasing. T

In [42]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [43]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

In [44]:
customers_arr[:5]

array([11000, 11001, 11002, 11003, 11004], dtype=int64)

In [45]:
get_items_purchased(11000, product_train, customers_arr, products_arr, item_lookup)

,StockCode,Description
5432,353,"Mountain-200 Silver, 38"
5442,214,"Sport-100 Helmet, Red"
5457,541,Touring Tire
5458,530,Touring Tire Tube
5459,573,"Touring-1000 Blue, 46"
5476,485,Fender Set - Mountain


In [46]:
from sklearn.preprocessing import MinMaxScaler

In [47]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Description.loc[item_lookup.StockCode == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'StockCode': codes, 'Description': descriptions}) # Create a dataframe 
    return final_frame[['StockCode', 'Description']] # Switch order of columns around

In [61]:
rec_items(11000, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)

,StockCode,Description
0,487,Hydration Pack - 70 oz.
1,358,"Mountain-200 Black, 38"
2,362,"Mountain-200 Black, 46"


In [49]:
get_items_purchased(11002, product_train, customers_arr, products_arr, item_lookup)

,StockCode,Description
1,346,"Mountain-100 Silver, 44"
5455,222,"Sport-100 Helmet, Blue"
5585,359,"Mountain-200 Black, 38"
5829,561,"Touring-1000 Yellow, 46"


In [50]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.772, 0.836)

In [56]:
rec_items(11002, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)

,StockCode,Description
0,463,"Half-Finger Gloves, S"
1,578,"Touring-2000 Blue, 50"
2,563,"Touring-1000 Yellow, 54"


In [57]:
get_items_purchased(11004, product_train, customers_arr, products_arr, item_lookup)

,StockCode,Description
81,345,"Mountain-100 Silver, 42"
5442,214,"Sport-100 Helmet, Red"
5476,485,Fender Set - Mountain
5509,355,"Mountain-200 Silver, 42"
5594,562,"Touring-1000 Yellow, 50"


In [58]:
rec_items(11004, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)

,StockCode,Description
0,353,"Mountain-200 Silver, 38"
1,360,"Mountain-200 Black, 42"
2,314,"Road-150 Red, 56"


In [59]:
get_items_purchased(11006, product_train, customers_arr, products_arr, item_lookup)

,StockCode,Description
1,346,"Mountain-100 Silver, 44"
5433,478,Mountain Bottle Cage
5594,562,"Touring-1000 Yellow, 50"


In [60]:
rec_items(11006, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)

,StockCode,Description
0,361,"Mountain-200 Black, 42"
1,353,"Mountain-200 Silver, 38"
2,363,"Mountain-200 Black, 46"
